In [ ]:
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
from multiprocessing import Pool
import datetime

In [ ]:
train=pd.read_csv('../train.csv')
test=pd.read_csv('../test_QyjYwdj.csv')
item=pd.read_csv('../item_data.csv')
transaction=pd.read_csv('../customer_transaction_data.csv')
coupon_item_mapping=pd.read_csv('../coupon_item_mapping.csv')
camp=pd.read_csv('../campaign_data.csv')
camp.start_date=pd.to_datetime(camp.start_date,format='%d/%m/%y')
camp.end_date=pd.to_datetime(camp.end_date,format='%d/%m/%y')
camp.sort_values('start_date',inplace=True)
df=pd.concat((train.iloc[:,:-1],test),axis=0)
df=df.merge(camp,on='campaign_id',how='left')
transaction.date=pd.to_datetime(transaction.date)

In [ ]:
temp=df[['customer_id','coupon_id','start_date','id','end_date']]
temp['cut_off_date']=pd.to_datetime(df.start_date) -datetime.timedelta(150)

In [ ]:
transaction=transaction.drop_duplicates()

In [ ]:
transaction.head()

In [ ]:
def get_feat(row):
    data_=row[1]
    temp=transaction[(transaction.item_id.isin(coupon_item_mapping[coupon_item_mapping.coupon_id==data_['coupon_id']].item_id))\
               &(transaction.date<data_['start_date'])]    
    
    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_.id]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_.id]+np.repeat(0,11).tolist()
    return feats

In [ ]:
%%time
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()

In [ ]:
cols=['id','num_bought_coup','num_discount_availed_coup','frac_discount_availed_coup','sum_discount_coup',
'selling_price_sum_coup',
'coupon_discount_mean_coup','selling_price_mean_coup','other_discount_sum_coup',
      'other_discount_mean_coup','cd_sp_ratio_mean_coup','cd_sp_ratio_sum_coup'
     ]
# cols=['id','num_bought_coup_end_date','num_discount_availed_coup_end_date','frac_discount_availed_coup_end_date',
#       'sum_discount_coup_end_date','selling_price_sum_coup_end_date']
feat_df=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

In [ ]:
feat_df.head()

In [ ]:
feat_df.to_csv('../coupon_item_feats_no_lag_v2.csv',index=False)

In [ ]:
def get_feat(row):
    data_=row[1]
#     temp=transaction[(transaction.customer_id==data_['customer_id'])&\
#                          (transaction.item_id.isin(coupon_item_mapping[coupon_item_mapping.coupon_id==data_['coupon_id']].item_id))\
#                &(transaction.date<data_['start_date'])]

    temp=transaction[(transaction.customer_id==data_['customer_id'])&\
                         (transaction.item_id.isin(coupon_item_mapping[coupon_item_mapping.coupon_id==data_['coupon_id']].item_id))\
               &(transaction.date<data_['start_date'])]
    
    
    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_.id]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_.id]+np.repeat(0,11).tolist()
    return feats

In [ ]:
# cols=['id','num_bought','num_discount_availed','frac_discount_availed','sum_discount','selling_price_sum']
cols=['id','num_bought','num_discount_availed','frac_discount_availed','sum_discount',
'selling_price_sum',
'coupon_discount_mean','selling_price_mean','other_discount_sum',
      'other_discount_mean','cd_sp_ratio_mean','cd_sp_ratio_sum'
     ]

In [ ]:
%%time
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()

In [ ]:
len(data_outputs)

In [ ]:
feat_df=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

In [ ]:
feat_df.to_csv('../feats_user_coup_item_no_lag_v2.csv',index=False)

In [ ]:
camp_cust=df.groupby(['campaign_id','customer_id'])['coupon_id'].unique().reset_index()

In [ ]:
camp_cust=camp_cust.merge(camp,on='campaign_id')

In [ ]:
%%time
def get_feat(row):
    data_=row[1]
    temp=transaction[(transaction.customer_id==data_['customer_id'])&\
                             (transaction.item_id.isin(coupon_item_mapping[coupon_item_mapping.coupon_id.isin(data_['coupon_id'])].item_id))\
                   &(transaction.date<data_['start_date'])]

    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_['campaign_id'],data_['customer_id']]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_['campaign_id'],data_['customer_id']]+np.repeat(0,11).tolist()

    return feats
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, camp_cust.iterrows())
finally: 
    pool.close()
    pool.join()
    
cols=['campaign_id','customer_id','num_bought_camp_cust','num_discount_availed_camp_cust','frac_discount_availed_camp_cust','sum_discount_camp_cust',
'selling_price_sum_camp_cust',
'coupon_discount_mean_camp_cust','selling_price_mean_camp_cust','other_discount_sum_camp_cust',
      'other_discount_mean_camp_cust','cd_sp_ratio_mean_camp_cust','cd_sp_ratio_sum_camp_cust'
     ]

camp_cust_hist=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

In [ ]:
camp_cust_hist.to_csv('../camp_cust_feats_v2.csv',index=False)

In [ ]:
a=pd.read_csv('../feats_user_coup_item_no_lag_v2.csv')

In [ ]:
b=pd.read_csv('../../av_amex/feats_user_coup_item_no_lag_v2.csv')